In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import datetime as dt
import time

plt.style.use('ggplot')
pd.set_option('display.max_rows', None)

In [ ]:
import talib as tb

In [ ]:
df = pd.read_csv('D:\TWF_Futures_Minute_Trade.txt')
df.head(3)

In [ ]:
df1 = pd.read_csv("C:/Users/thoma/Downloads/TXF_樣本外 (1).csv")
df1.tail(3)

In [ ]:
df.index = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.drop(columns=['Date','Time'])
df.columns = ['open', 'high', 'low', 'close','volume']
df['Hour'] = df.index.map(lambda x: x.hour)
df.head(3)

In [ ]:
df1.index = pd.to_datetime(df1['Date'] + ' ' + df1['Time'])
df1 = df1.drop(columns=['Date','Time'])
df1.columns = ['open', 'high', 'low', 'close','volume']
df1['Hour'] = df1.index.map(lambda x: x.hour)
df1.head(3)

In [ ]:
df2 = pd.concat([df, df1], axis=0)
df2.tail(3)

In [ ]:
df=df2
df.tail(3)

In [ ]:
df.info()

In [ ]:
plt.figure(figsize = (15,7))
plt.plot(df['close'], color = 'red', alpha = 0.5, label = 'Close price')
plt.plot(df['volume'], color = '#01889f', alpha = 0.5, label = 'Volume')
plt.legend(loc='upper left')
plt.title('Volume History');

In [ ]:
rule = '60T'

Morning = df[(df['Hour'] >= 8) & (df['Hour'] <= 13)]
Morning.index = Morning.index + dt.timedelta(minutes=15)

d1 = Morning.resample(rule=rule, closed='right', label='left').first()[['open']]
d2 = Morning.resample(rule=rule, closed='right', label='left').max()[['high']]
d3 = Morning.resample(rule=rule, closed='right', label='left').min()[['low']]
d4 = Morning.resample(rule=rule, closed='right', label='left').last()[['close']]
d5 = Morning.resample(rule=rule, closed='right', label='left').sum()[['volume']]

df_Morning = pd.concat([d1,d2,d3,d4,d5], axis=1)
df_Morning = df_Morning.dropna()
df_Morning.index = df_Morning.index - dt.timedelta(minutes=15)
df_Morning.head()

In [ ]:
rule = '60T'

Night = df[(df['Hour'] < 8) | (df['Hour'] > 13)]

d1 = Night.resample(rule=rule, closed='right', label='left').first()[['open']]
d2 = Night.resample(rule=rule, closed='right', label='left').max()[['high']]
d3 = Night.resample(rule=rule, closed='right', label='left').min()[['low']]
d4 = Night.resample(rule=rule, closed='right', label='left').last()[['close']]
d5 = Night.resample(rule=rule, closed='right', label='left').sum()[['volume']]

df_Night = pd.concat([d1,d2,d3,d4,d5], axis=1)
df_Night = df_Night.dropna()
df_Night.head()

In [ ]:
df_Day = pd.concat([df_Morning, df_Night], axis=0)
df_Day = df_Day.sort_index(ascending=True)
df_Day.head()

In [ ]:
df_Day['Hour'] = df_Day.index.map(lambda x: x.hour)

In [ ]:
AllData = df_Day[(df_Day.index >= '2011-01-01 00:00:00') & (df_Day.index <= '2023-7-1 00:00:00')].copy()

In [ ]:
AllData.info()

In [ ]:
trainData = df_Day[(df_Day.index >= '2011-01-01 00:00:00') & (df_Day.index <= '2019-12-31 00:00:00')].copy()
testData = df_Day[(df_Day.index >= '2020-1-1 00:00:00') & (df_Day.index <= '2023-7-1 00:00:00')].copy()

In [ ]:
trainData.head(3)

In [ ]:
testData.tail(3)

In [ ]:
settlementDate_ = pd.read_csv("D:/結算日.csv")
settlementDate_.columns = ['settlementDate', 'futures', 'settlementPrice']
settlementDate_

In [ ]:
bool_ = [False if 'W' in i else True for i in settlementDate_['futures']]

In [ ]:
len(list(settlementDate_[bool_]['futures']))

In [ ]:
settlementDate = [i.replace('/','-') for i in list(settlementDate_[bool_]['settlementDate'])]

In [ ]:
settlementDate = [pd.to_datetime(i).date() for i in settlementDate]

In [ ]:
settlementDate[0:5]

In [ ]:
#參數設定
fund = 1000000
feePaid = 600
length = 15
NumStd = 2.5
da = 8
K = 0.04


#指標計算
# trainData['H'] = trainData['high'].shift(1).rolling(window=length, center=False).max()
# trainData['L'] = trainData['low'].shift(1).rolling(window=length, center=False).min()
AllData["mid price"] = AllData['close'].rolling(window=length, center=False).mean()
AllData["ATR"] = tb.ATR(AllData['high'],AllData['low'],AllData['close'],timeperiod = da)
AllData['upLine'] = AllData["mid price"] + NumStd*AllData['ATR']
AllData['downLine'] = AllData["mid price"] - NumStd*AllData['ATR']
AllData["mo"] = ( AllData["close"] - AllData["close"].shift(da) )/ AllData["close"].shift(da)
#trainData["EMA200"] = trainData['close'].ewm(span=200, adjust=False).mean()
#trainData['RSI'] = abstract.RSI(trainData, timeperiod = da)
#trainData['cci'] = tb.CCI(trainData['high'], trainData['low'], trainData['close'], timeperiod= da)
AllData.tail(6)

In [ ]:
#參數設定
fund = 1000000
feePaid = 600
length = 15
NumStd = 2.5
da = 8
K = 0.04


#指標計算
# trainData['H'] = trainData['high'].shift(1).rolling(window=length, center=False).max()
# trainData['L'] = trainData['low'].shift(1).rolling(window=length, center=False).min()
trainData["mid price"] = trainData['close'].rolling(window=length, center=False).mean()
trainData["ATR"] = tb.ATR(trainData['high'],trainData['low'],trainData['close'],timeperiod = da)
trainData['upLine'] = trainData["mid price"] + NumStd*trainData['ATR']
trainData['downLine'] = trainData["mid price"] - NumStd*trainData['ATR']
trainData["mo"] = ( trainData["close"] - trainData["close"].shift(da) )/ trainData["close"].shift(da)
#trainData["EMA200"] = trainData['close'].ewm(span=200, adjust=False).mean()
#trainData['RSI'] = abstract.RSI(trainData, timeperiod = da)
#trainData['cci'] = tb.CCI(trainData['high'], trainData['low'], trainData['close'], timeperiod= da)
trainData.tail(6)

In [ ]:
#參數設定
fund = 1000000
feePaid = 600
length = 15
NumStd = 2.5
da = 8
K = 0.04


#指標計算
# trainData['H'] = trainData['high'].shift(1).rolling(window=length, center=False).max()
# trainData['L'] = trainData['low'].shift(1).rolling(window=length, center=False).min()
testData["mid price"] = testData['close'].ewm(span=length, adjust=False).mean()
testData["ATR"] = tb.ATR(testData['high'],testData['low'],testData['close'],timeperiod = da)
testData['upLine'] = testData["mid price"] + NumStd*testData['ATR']
testData['downLine'] = testData["mid price"] - NumStd*testData['ATR']
testData["mo"] = ( testData["close"] - testData["close"].shift(da) )/ testData["close"].shift(da)
#testData["EMA200"] = testData['close'].ewm(span=200, adjust=False).mean()

#testData['cci'] = tb.CCI(testData['high'], testData['low'], testData['close'], timeperiod= da)

#testData["mo"] = ( testData["close"] - testData["close"].shift(da) )/ testData["close"].shift(da)
testData.tail(3)

In [ ]:
df_arr = np.array(AllData)
time_arr = np.array(AllData.index)
date_arr = [pd.to_datetime(i).date() for i in time_arr]

#翻譯蒟蒻
# date_arr = []
# for i in time_arr:
#     date_arr.append(pd.to_datetime(i).date())

In [ ]:
df_arr = np.array(trainData)
time_arr = np.array(trainData.index)
date_arr = [pd.to_datetime(i).date() for i in time_arr]

#翻譯蒟蒻
# date_arr = []
# for i in time_arr:
#     date_arr.append(pd.to_datetime(i).date())

In [ ]:
df_arr = np.array(testData)
time_arr = np.array(testData.index)
date_arr = [pd.to_datetime(i).date() for i in time_arr]

#翻譯蒟蒻
# date_arr = []
# for i in time_arr:
#     date_arr.append(pd.to_datetime(i).date())

In [ ]:
#建立存放資料的單位
BS = None
buy = []
sell = []
profit_list = [0]
profit_fee_list = [0]
profit_fee_list_realized = []

#開始交易流程
for i in range(len(df_arr)):

    #回測期間最後一天就跳出這個迴圈
    if i == len(df_arr)-1:
        break

    ## 進場邏輯
    ### 當收盤價突破布林上軌
    entryLong = df_arr[i, 3] > df_arr[i,8] and df_arr[i-1, 3] < df_arr[i-1, 8] and df_arr[i, 10] > df_arr[i-1, 10] > 0
    ### 結算日不進場
    entryCondition = date_arr[i] not in settlementDate

    ## 出場邏輯
    ### 當收盤價跌破布林下軌
    exitShort = df_arr[i, 3] < df_arr[i,8] and df_arr[i, 10]  < 0 < df_arr[i-1, 10]
    ### 結算日時放到結算
    exitCondition = date_arr[i] in settlementDate and df_arr[i,5] >= 11

    #做多的狀態
    if BS == 'B':
        stopLoss = df_arr[i,3] <= df_arr[t,0] * (1-K)
        stopProfit = df_arr[i,3] >= df_arr[t,0] * (1+K)
    if exitCondition == True:
         print(f'{time_arr[i]}')

    #還沒進場不用計算損益
    if BS == None:
        profit_list.append(0)
        profit_fee_list.append(0)

        #確認進場&相關設定
        if entryLong and entryCondition:
            #更改狀態至做多
            BS = 'B'
            #紀錄進場時間
            t = i+1
            buy.append(t)
            print("Buy Price: {}, time: {}".format(df_arr[t,0], time_arr[t]))

    #進場開始計算未實現損益
    elif BS == 'B':

        profit = 200 * (df_arr[i+1,0] - df_arr[i,0])
        profit_list.append(profit)

        #近場條件達成，計算未實現損益-交易成本
        if exitShort or i == len(df_arr)-2 or exitCondition or stopLoss or stopProfit:
            pl_round = 200 * (df_arr[i+1,0] - df_arr[t,0])
            profit_fee = profit - feePaid*2
            profit_fee_list.append(profit_fee)

            #紀錄出場時間
            sell.append(i+1)

            #重置交易狀態
            BS = None
            print("Sell Price: {}, time: {}".format(df_arr[i+1,0], time_arr[i+1]))
            print("Trade completed")
            print()


            # Realized PnL
            profit_fee_realized = pl_round - feePaid*2
            profit_fee_list_realized.append(profit_fee_realized)

        #出場條件未達成，計算未實現損益
        else:
            profit_fee = profit
            profit_fee_list.append(profit_fee)


equity = pd.DataFrame({'profit':np.cumsum(profit_list), 'profitfee':np.cumsum(profit_fee_list)}, index=AllData.index)
# print(equity)
equity.plot(grid=True, figsize=(12,6));

In [ ]:
BS = None
sellshort = []
buytocover = []
profit_list = [0]
profit_fee_list = [0]
profit_fee_list_realized = []

for i in range(len(df_arr)):

    if i == len(df_arr)-1:
        break

    ## 進場邏輯
    entrySellShort = df_arr[i, 3] < df_arr[i,9] and df_arr[i-1, 3] > df_arr[i-1, 9] and 0 > df_arr[i-1, 10] > df_arr[i, 10]
    entryCondition = date_arr[i] not in settlementDate

    ## 出場邏輯
    exitBuyToCover = df_arr[i, 3] > df_arr[i,9] and  df_arr[i-1, 10] < 0 < df_arr[i, 10]
    exitCondition = date_arr[i] in settlementDate and df_arr[i,5] >= 11

    if BS == 'S':
        stopLoss = df_arr[i,3] >= df_arr[t,0] * (1+K)
        stopProfit = df_arr[i,3] <= df_arr[t,0] * (1-K)

#     if exitCondition == True:
#         print(f'{time_arr[i]}')

    if BS == None:
        profit_list.append(0)
        profit_fee_list.append(0)

        if entrySellShort and entryCondition:
            BS = 'S'
            t = i+1
            sellshort.append(t)
            print("sell short at {}".format(time_arr[t]))


    elif BS == 'S':
        profit = 200 * (df_arr[i,0] - df_arr[i+1,0])
        profit_list.append(profit)

        if exitBuyToCover or i == len(df_arr)-2 or exitCondition or stopLoss or stopProfit:
            pl_round = 200 * (df_arr[t,0] - df_arr[i+1,0])
            profit_fee = profit - feePaid*2
            profit_fee_list.append(profit_fee)
            buytocover.append(i+1)
            BS = None
            print("buytocover at {}".format(time_arr[i+1]))
            print("trade completed")
            print()

            # Realized PnL
            profit_fee_realized = pl_round - feePaid*2
            profit_fee_list_realized.append(profit_fee_realized)

        else:
            profit_fee = profit
            profit_fee_list.append(profit_fee)


equity = pd.DataFrame({'profit':np.cumsum(profit_list), 'profitfee':np.cumsum(profit_fee_list)}, index=AllData.index)
# print(equity)
equity.plot(grid=True, figsize=(12,6));

In [ ]:
BS = None
buy = []
sell = []
sellshort = []
buytocover = []
profit_list = [0]
profit_fee_list = [0]
profit_fee_list_realized = []
rets = []

for i in range(len(df_arr)):

    if i == len(df_arr)-1:
        break

    ## 進場邏輯
    entryLong =  df_arr[i, 3] > df_arr[i,8] and df_arr[i-1, 3] < df_arr[i-1, 8] and df_arr[i, 10] > df_arr[i-1, 10] > 0
    entrySellShort =  df_arr[i, 3] < df_arr[i,9] and df_arr[i-1, 3] > df_arr[i-1, 9] and df_arr[i, 10] < df_arr[i-1, 10] < 0
    entryCondition = date_arr[i] not in settlementDate

                    ## 出場邏輯
    exitShort = df_arr[i, 3] < df_arr[i,8] and df_arr[i, 10]  < 0 < df_arr[i-1, 10]
    exitBuyToCover = df_arr[i, 3] > df_arr[i,9] and df_arr[i, 10]  > 0 > df_arr[i-1, 10] 
    exitCondition = date_arr[i] in settlementDate and df_arr[i,5] >= 11

                    ## 停利停損邏輯
    if BS == 'B':
        stopLoss = df_arr[i,3] <= df_arr[t,0] * (1-K)
        stopProfit = df_arr[i,3] >= df_arr[t,0] * (1+K)
    elif BS == 'S':
        stopLoss = df_arr[i,3] >= df_arr[t,0] * (1+K)
        stopProfit = df_arr[i,3] <= df_arr[t,0] * (1-K)


#     if exitCondition == True:
#         print(f'{time_arr[i]}')

    if BS == None:
        profit_list.append(0)
        profit_fee_list.append(0)

        if entryLong and entryCondition:
            BS = 'B'
            t = i+1
            buy.append(t)
            print("Enter Long Position")
            print("Buy Price: {}, time: {}".format(df_arr[t,0], time_arr[t]))

        elif entrySellShort and entryCondition:
            BS = 'S'
            t = i+1
            sellshort.append(t)
            print("Enter Short Position")
            print("Sell Price: {}, time: {}".format(df_arr[t,0], time_arr[t]))

    elif BS == 'B':
        profit = 200 * (df_arr[i+1,0] - df_arr[i,0])
        profit_list.append(profit)

        if exitShort or i == len(df_arr)-2 or exitCondition or stopLoss or stopProfit:
            pl_round = 200 * (df_arr[i+1,0] - df_arr[t,0])
            profit_fee = profit - feePaid*2
            profit_fee_list.append(profit_fee)
            sell.append(i+1)
            BS=None
            print("Sell Price: {}, time: {}".format(df_arr[i+1,0], time_arr[i+1]))
            print("Trade completed")
            print()

            # Realized PnL
            profit_fee_realized = pl_round - feePaid*2
            profit_fee_list_realized.append(profit_fee_realized)
            rets.append(profit_fee_realized/(200*df_arr[t,0]))

        else:
            profit_fee = profit
            profit_fee_list.append(profit_fee)

    elif BS == 'S':
        profit = 200 * (df_arr[i,0] - df_arr[i+1,0])
        profit_list.append(profit)

        if exitBuyToCover or i == len(df_arr)-2 or exitCondition or stopLoss or stopProfit:
            pl_round = 200 * (df_arr[t,0] - df_arr[i+1,0])
            profit_fee = profit - feePaid*2
            profit_fee_list.append(profit_fee)
            buytocover.append(i+1)
            BS = None
            print("Buycover Price: {}, time: {}".format(df_arr[i+1,0], time_arr[i+1]))
            print("Trade completed")
            print()

            # Realized PnL
            profit_fee_realized = pl_round - feePaid*2
            profit_fee_list_realized.append(profit_fee_realized)
            rets.append(profit_fee_realized/(200*df_arr[t,0]))

        else:
            profit_fee = profit
            profit_fee_list.append(profit_fee)


equity = pd.DataFrame({'profit':np.cumsum(profit_list), 'profitfee':np.cumsum(profit_fee_list)}, index=AllData.index)
# print(equity)
equity.plot(grid=True, figsize=(12,6));

In [ ]:
print(buy)
print(sell)
print(sellshort)
print(buytocover)

In [ ]:
equity.iloc[-3:]

In [ ]:
equity['equity'] = equity['profitfee'] + fund
equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
equity['drawdown'] = equity['equity'] - equity['equity'].cummax()

In [ ]:
equity.iloc[-3:]

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))

high_index = equity[equity['profitfee'].cummax() == equity['profitfee']].index
equity['profitfee'].plot(label = 'Total Profit', ax = ax, c = 'r', grid=True)
plt.fill_between(equity['drawdown'].index, equity['drawdown'], 0, facecolor  = 'r', label = 'Drawdown', alpha=0.5)
plt.scatter(high_index, equity['profitfee'].loc[high_index],c = '#02ff0f', label = 'High')

plt.legend()
plt.ylabel('Accumulated Profit')
plt.xlabel('Time')
plt.title('Profit & drawdown',fontsize  = 16);

In [ ]:
profit = equity['profitfee'].iloc[-1]
ret = equity['equity'][-1]/equity['equity'][0] - 1
mdd = abs(equity['drawdown_percent'].min())
calmarRatio = ret / mdd
tradeTimes = len(buy)+len(sellshort)
winRate = len([i for i in profit_fee_list_realized if i > 0]) / len(profit_fee_list_realized)
profitFactor = sum([i for i in profit_fee_list_realized if i > 0]) / abs(sum([i for i in profit_fee_list_realized if i < 0]))
winLossRatio = np.mean([i for i in profit_fee_list_realized if i > 0]) / abs(np.mean([i for i in profit_fee_list_realized if i < 0]))

print(f'profit: ${np.round(profit,2)}')
print(f'return: {np.round(ret,4)*100}%')
print(f'mdd: {np.round(mdd,4)*100}%')
print(f'calmarRatio: {np.round(calmarRatio,2)}')
print(f'tradeTimes: {tradeTimes}')
print(f'winRate: {np.round(winRate,4)*100}%')
print(f'profitFactor: {np.round(profitFactor,2)}')
print(f'winLossRatio: {np.round(winLossRatio,2)}')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as py
plt.style.use('ggplot')

In [ ]:
equity['equity'] = equity['profitfee'] + fund
equity['drawdown_percent'] = (equity['equity']/equity['equity'].cummax()) - 1
equity['drawdown'] = equity['equity'] - equity['equity'].cummax() #前n個元素的最大值
#print(profit_fee_list_realized)
profit = equity['profitfee'].iloc[-1]
ret = equity['equity'][-1]/equity['equity'][0] - 1
mdd = abs(equity['drawdown_percent'].min())
calmarRatio = ret / mdd
tradeTimes = len(sellshort) + len(buy) 
winRate = len([i for i in profit_fee_list_realized if i > 0]) / len(profit_fee_list_realized)
profitFactor = sum([i for i in profit_fee_list_realized if i>0]) / abs(sum([i for i in profit_fee_list_realized if i<0]))

print('Profit : ',profit)
print('Return : ',ret)
print('Max DrawDown : ',mdd)
print('Caimar Ratio : ',calmarRatio)
print('Trade Times : ',tradeTimes)
print('Win Rate : ',winRate)
print('Profit Factor : ',profitFactor)

In [ ]:
equity.info()

In [ ]:
pct_change = equity["equity"].pct_change()
retur = pct_change.mean()
std = pct_change.std()
print(retur,std)

In [ ]:
def calc_performance_index_func(percent_return_series, days_of_year=365):

    if str(percent_return_series.index.freq)!='<Day>':
        percent_return_series=percent_return_series.resample('1d').sum()

    cum_return_percent_data=percent_return_series.cumsum()

    mdd=(cum_return_percent_data.cummax()-cum_return_percent_data).max()

    return_std=(percent_return_series).std()
    return_downside_std=(percent_return_series * (percent_return_series<0)).std()
    if return_std!=0:
        sharpe_ratio=round(percent_return_series.mean()/return_std*(days_of_year**(1/2)),2)
    else:
        sharpe_ratio=None

    if return_downside_std!=0:
        sortino_ratio=round(percent_return_series.mean()/return_downside_std*(days_of_year**(1/2)),2)
    else:
        sortino_ratio=None
        
    if mdd!=0:
        rtomdd=round((cum_return_percent_data[-1])/mdd,2)
        calmar_ratio=round((rtomdd/len(percent_return_series))*days_of_year,2)
    else:
        rtomdd=None
        calmar_ratio=None

    performance_index_dict={
        "Cumulative Return":cum_return_percent_data[-1],
        "Sharpe Ratio":sharpe_ratio,
        "Sortino Ratio":sortino_ratio,
        "Calmar Ratio":calmar_ratio,
        "Return_MDD": rtomdd,
        "MDD":round(mdd,2)
    }
    return performance_index_dict

In [ ]:
calc_performance_index_func(pct_change , days_of_year=365)

In [ ]:
# 時間損益(年)
equity.index = pd.to_datetime(equity.index) #確保索引是datetime型態
years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019','2020','2021','2022','2023']
year_ret = []
print(equity.index.year.unique())
for i in equity.index.year.unique():
    year_ret.append(equity[str(i)]['equity'].iloc[-1]/equity[str(i)]['equity'].iloc[0] - 1)

df = pd.DataFrame({'Return':year_ret},index = years)

# heatmap函式
py.figure(figsize=(10,1))
sns.heatmap(df.transpose(), annot=True, cmap='OrRd')
py.title('Return by year')
py.show()
print('')

# 時間損益(月)
group_data = equity.groupby([equity.index.year, equity.index.month])

month_ret = []
for group, data in group_data:
    month_ret.append(data['equity'].iloc[-1]/data['equity'].iloc[0] - 1)

df = pd.DataFrame(np.array(month_ret).reshape((13,12)), columns=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                    index = years)

print(df)
py.figure(figsize=(15,15))
sns.heatmap(df, annot=True, cmap='OrRd')
py.title('Return by month')
py.show()

In [ ]:
# 損益分布
print(rets)
sns.histplot(rets, kde=True)
py.xlabel('Return per trade')
py.ylabel('Frequency')
py.title('Profit Loss Distribution')
py.show()

In [ ]:
# quantstats回測報表
!pip install quantstats
import quantstats
ret = equity['equity'].pct_change(periods=1).dropna()
print(ret)
quantstats.reports.full(ret)
quantstats.reports.html(ret,output='stats.html',title='backtest result',download_filename='result.html')

In [ ]:
quantstats.plots.snapshot(equity,title = "Performance")

In [ ]:
optimizationList = []
fund = 1000000
feePaid = 600
length = 20
NumStd = 1.5
da = 19
K = 0.04

# 最佳化參數：length & NumStd
for length in range(10,50,5):
    for NumStd  in np.arange(1, 3, 0.5):
        NumStd = np.round(NumStd ,2)
        for da in range(7,15,1):
                
            print('----------')
            print(f'length: {length}')
            print(f'NumStd : {NumStd }')
            print(f'da: {da}')

            trainData["mid price"] = trainData['close'].rolling(window=length, center=False).mean()
            trainData["ATR"] = tb.ATR(trainData['high'],trainData['low'],trainData['close'],timeperiod = da)
            trainData['upLine'] = trainData["mid price"] + NumStd*trainData['ATR']
            trainData['downLine'] = trainData["mid price"] - NumStd*trainData['ATR']
            trainData["mo"] = ( trainData["close"] - trainData["close"].shift(da) )/ trainData["close"].shift(da)

                #df['ma'] = df['close'].rolling(window=length, center=False).mean()
                #df['std'] = df['close'].rolling(window=length, center=False).std()

            df_arr = np.array(trainData)
            time_arr = np.array(trainData.index)
            date_arr = [pd.to_datetime(i).date() for i in time_arr]

            BS = None
            buy = []
            sell = []
            sellshort = []
            buytocover = []
            profit_list = [0]
            profit_fee_list = [0]
            profit_fee_list_realized = []
            rets = []

            for i in range(len(df_arr)):

                if i == len(df_arr)-1:
                    break

                    ## 進場邏輯
                entryLong =  df_arr[i, 3] > df_arr[i,8] and df_arr[i-1, 3] < df_arr[i-1, 8] and df_arr[i, 10] > df_arr[i-1, 10] > 0
                entrySellShort =  df_arr[i, 3] < df_arr[i,9] and df_arr[i-1, 3] > df_arr[i-1, 9] and df_arr[i, 10] < df_arr[i-1, 10] < 0
                entryCondition = date_arr[i] not in settlementDate

                    ## 出場邏輯
                exitShort = df_arr[i, 3] < df_arr[i,8] and df_arr[i, 10]  < 0 < df_arr[i-1, 10]
                exitBuyToCover = df_arr[i, 3] > df_arr[i,9] and df_arr[i, 10]  > 0 > df_arr[i-1, 10] 
                exitCondition = date_arr[i] in settlementDate and df_arr[i,5] >= 11

                    ## 停利停損邏輯
                if BS == 'B':
                    stopLoss = df_arr[i,3] <= df_arr[t,0] * (1-K)
                    stopProfit = df_arr[i,3] >= df_arr[t,0] * (1+K)
                elif BS == 'S':
                    stopLoss = df_arr[i,3] >= df_arr[t,0] * (1+K)
                    stopProfit = df_arr[i,3] <= df_arr[t,0] * (1-K)

                    #     if exitCondition == True:
                    #         print(f'{time_arr[i]}')

                if BS == None:
                    profit_list.append(0)
                    profit_fee_list.append(0)

                    if entryLong and entryCondition:
                        BS = 'B'
                        t = i+1
                        buy.append(t)

                    elif entrySellShort and entryCondition:
                        BS = 'S'
                        t = i+1
                        sellshort.append(t)

                elif BS == 'B':
                    profit = 200 * (df_arr[i+1,0] - df_arr[i,0])
                    profit_list.append(profit) #紀錄損益

                    if exitShort or i == len(df_arr)-2 or exitCondition or stopLoss or stopProfit:
                        pl_round = 200 * (df_arr[i+1,0] - df_arr[t,0])
                        profit_fee = profit - feePaid*2
                        profit_fee_list.append(profit_fee)
                        sell.append(i+1)
                        BS=None

                            # Realized PnL
                        profit_fee_realized = pl_round - feePaid*2
                        profit_fee_list_realized.append(profit_fee_realized)
                        rets.append(profit_fee_realized/(200*df_arr[t,0]))

                    else:
                        profit_fee = profit
                        profit_fee_list.append(profit_fee)

                elif BS == 'S':
                    profit = 200 * (df_arr[i,0] - df_arr[i+1,0])
                    profit_list.append(profit)

                    if exitBuyToCover or i == len(df_arr)-2 or exitCondition or stopLoss or stopProfit:
                        pl_round = 200 * (df_arr[t,0] - df_arr[i+1,0])
                        profit_fee = profit - feePaid*2
                        profit_fee_list.append(profit_fee)
                        buytocover.append(i+1)
                        BS=None

                            # Realized PnL
                        profit_fee_realized = pl_round - feePaid*2
                        profit_fee_list_realized.append(profit_fee_realized)
                        rets.append(profit_fee_realized/(200*df_arr[t,0]))

                    else:
                        profit_fee = profit
                        profit_fee_list.append(profit_fee)

            equity = pd.DataFrame({'profit':np.cumsum(profit_list), 'profitfee':np.cumsum(profit_fee_list)}, index=trainData.index)
            equity['equity'] = equity['profitfee'] + fund
            equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
            equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
            ret = equity['equity'][-1]/equity['equity'][0] - 1
            mdd = abs(equity['drawdown_percent'].min())
            calmarRatio = ret / mdd
            winRate = len([i for i in profit_fee_list_realized if i > 0]) / len(profit_fee_list_realized)
            profit = equity['profitfee'].iloc[-1]

            optimizationList.append([length, NumStd ,da,ret, calmarRatio])

print(optimizationList)

In [ ]:
optResult = pd.DataFrame(optimizationList, columns=[ "length",'NumStd','da','ret','calmarRatio'])
optResult

In [ ]:
optResult[(optResult['calmarRatio'] > 1)].sort_values('ret', ascending=False).iloc[:]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(equity.index, equity['equity'], label='Equity Curve', color='b')
plt.xlabel('Date')
plt.ylabel('Equity Value')
plt.title('Equity Curve')
plt.grid(True)
plt.legend()
plt.show()